In [ ]:
### FEATURE CALCULATION
### May 23, 2025
### Lorena Lopez Dominguez
### From https://brainconn.readthedocs.io/en/latest/api.html
### kernel: base (Python 3.12.8)

In [18]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import bct

In [30]:
# Set working directory
base_dir = '/Users/Lore/Desktop/brainhack/project/BrainHack-Project-'
data_dir = os.path.join(base_dir, 'Data')
os.chdir(base_dir)

# Create results directory if it doesn't exist
results_dir = os.path.join(base_dir, 'results')
os.makedirs(results_dir, exist_ok=True)

In [31]:
# Load community index vector
civ = pd.read_table('network_roi_7networks.txt')

In [32]:
# Load metadata
meta = pd.read_csv(os.path.join('/Users/Lore/Desktop/brainhack/project/demographic_adhd_controls.csv'))
meta

# Create dictionary mapping subject ID to group ('case' or 'control')
group_map = dict(zip(meta['ID'], meta['MHAM_85']))

In [33]:
# Initialize DataFrames to store all subjects' data
all_zscores = pd.DataFrame()
all_partcoef = pd.DataFrame()
all_strength = pd.DataFrame()

In [34]:
# Initialize accumulators for average plots, separate for cases and controls
sum_wei_adhd = None
sum_bin_adhd = None
count_adhd = 0

sum_wei_ctrl = None
sum_bin_ctrl = None
count_ctrl = 0

In [42]:
# Loop through files in directory
for filename in os.listdir(data_dir):
    if filename.startswith('data_sub-') and filename.endswith('.txt'):
        subject_name = re.search(r'data_sub-(.*)\.txt', filename).group(1)
        file_path = os.path.join(data_dir, filename)

        # Load FC data
        FCData = np.loadtxt(file_path, delimiter=',', skiprows=1)

        # Transform to weighted and binary adjacency matrices
        adj_wei = FCData - np.eye(FCData.shape[0])
        adj_bin = bct.utils.binarize(bct.utils.threshold_proportional(adj_wei, 0.2))
        
        # Determine group
        group = group_map.get(subject_name)
        
        if group == 1:
            if sum_wei_adhd is None:
                sum_wei_adhd = adj_wei.copy()
                sum_bin_adhd = adj_bin.copy()
            else:
                sum_wei_adhd += adj_wei
                sum_bin_adhd += adj_bin
            count_adhd += 1

        elif group == 0:
            if sum_wei_ctrl is None:
                sum_wei_ctrl = adj_wei.copy()
                sum_bin_ctrl = adj_bin.copy()
            else:
                sum_wei_ctrl += adj_wei
                sum_bin_ctrl += adj_bin
            count_ctrl += 1

        # Calculate centrality measures
        mod_z = bct.centrality.module_degree_zscore(adj_wei, civ.network_group)
        part_coef = bct.centrality.participation_coef(adj_wei, civ.network_group)
        strength = bct.degree.strengths_und(adj_wei)

        # Store in DataFrames with subject name as column
        all_zscores[subject_name] = mod_z
        all_partcoef[subject_name] = part_coef
        all_strength[subject_name] = strength

        # # Optional: Save plots per subject (can comment out if not needed)
        # fig, ax = plt.subplots(figsize=(7, 7))
        # ax.imshow(adj_wei, cmap='viridis')
        # plt.title(f'Weighted adjacency matrix: {subject_name}')
        # fig.colorbar(ax.images[0])
        # plt.savefig(os.path.join(data_dir, f'{subject_name}_plot_wei.png'))
        # plt.close()

        # fig, ax = plt.subplots(figsize=(7, 7))
        # ax.imshow(adj_bin, cmap='viridis')
        # plt.title(f'Binary adjacency matrix: {subject_name}')
        # fig.colorbar(ax.images[0])
        # plt.savefig(os.path.join(data_dir, f'{subject_name}_plot_bin.png'))
        # plt.close()

In [43]:
# Transpose to have subjects as rows
z_df = all_zscores.T.reset_index()
pc_df = all_partcoef.T.reset_index()
strength_df = all_strength.T.reset_index()

# Rename the index column to 'ID'
z_df.rename(columns={'index': 'ID'}, inplace=True)
pc_df.rename(columns={'index': 'ID'}, inplace=True)
strength_df.rename(columns={'index': 'ID'}, inplace=True)

z_df.columns = ['ID'] + [f'ROI_{i+1}' for i in range(z_df.shape[1] - 1)]
pc_df.columns = ['ID'] + [f'ROI_{i+1}' for i in range(pc_df.shape[1] - 1)]
strength_df.columns = ['ID'] + [f'ROI_{i+1}' for i in range(strength_df.shape[1] - 1)]


In [27]:
# Save all features to CSV
z_df.to_csv(os.path.join(results_dir, 'within_module_z_all.csv'), index=False)
pc_df.to_csv(os.path.join(results_dir, 'participation_coeff_all.csv'), index=False)
strength_df.to_csv(os.path.join(results_dir, 'node_strength_all.csv'), index=False)


In [44]:
# Compute group-specific averages
avg_wei_adhd = sum_wei_adhd / count_adhd
avg_bin_adhd = sum_bin_adhd / count_adhd
avg_wei_ctrl = sum_wei_ctrl / count_ctrl
avg_bin_ctrl = sum_bin_ctrl / count_ctrl

# Plot weighted and binary for cases
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(avg_wei_adhd, cmap='viridis')
plt.title('ADHD: Average Weighted Adjacency')
fig.colorbar(ax.imshow(avg_wei_adhd, cmap='viridis'))
plt.savefig(os.path.join(results_dir, 'adhd_avg_weighted_adjacency.png'))
plt.close()

fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(avg_bin_adhd, cmap='viridis')
plt.title('ADHD: Average Binary Adjacency')
fig.colorbar(ax.imshow(avg_bin_adhd, cmap='viridis'))
plt.savefig(os.path.join(results_dir, 'adhd_avg_binary_adjacency.png'))
plt.close()

# Plot weighted and binary for controls
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(avg_wei_ctrl, cmap='viridis')
plt.title('Control: Average Weighted Adjacency')
fig.colorbar(ax.imshow(avg_wei_ctrl, cmap='viridis'))
plt.savefig(os.path.join(results_dir, 'control_avg_weighted_adjacency.png'))
plt.close()

fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(avg_bin_ctrl, cmap='viridis')
plt.title('Control: Average Binary Adjacency')
fig.colorbar(ax.imshow(avg_bin_ctrl, cmap='viridis'))
plt.savefig(os.path.join(results_dir, 'control_avg_binary_adjacency.png'))
plt.close()
